Integrantes:
- Sebastian Lopez
- Mateo Tapia

In [27]:
!pip -q install ultralytics # Instalamos Ultralytics (nos permitirá usar YOLO)

pyenv: pip: command not found

The `pip' command exists in these Python versions:
  3.12.11
  3.13.7

Note: See 'pyenv help global' for tips on allowing both
      python2 and python3 to be found.


In [28]:
!unzip -o datasets.zip

Archive:  datasets.zip
  inflating: datasets/BCCD/labels/train/BloodImage_00339.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00287.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00107.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00305.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00089.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00311.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00199.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00193.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00204.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00077.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00083.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00247.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00212.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00340.txt  
  inflating: datasets/BCCD/labels/train/BloodImage_00264.txt  
  inflating: datasets/BCCD/label

In [29]:
from ultralytics import YOLO
from pathlib import Path
import pandas as pd
import itertools

model_coco = YOLO("yolov8n.pt")

In [30]:
param_grid = {
    # Hiperparámetros a variar
    'freeze': [22, 10, 0],          # 22 (base), 10 (medio), 0 (all entrenable)
    'optimizer': ['SGD', 'AdamW'],  # AdamW suele converger más rápido
    'lr0': [0.01, 0.001],           # Learning rate inicial
    'batch': [8, 16, 32],              # Probamos 16 y 32.
}


In [31]:
BASE_PATH = Path("/home/s.lopezmedina/Maestria/4_computer_vision/codigo/8_laboratorio_calificado_4/")
OUTPUT_DIR = BASE_PATH.joinpath("runs","detect")

In [32]:
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"Total de experimentos a correr: {len(combinations)}")
print(f"Los resultados se guardarán en: {OUTPUT_DIR}")

Total de experimentos a correr: 36
Los resultados se guardarán en: /home/s.lopezmedina/Maestria/4_computer_vision/codigo/8_laboratorio_calificado_4/runs/detect


In [33]:
all_results = []

for i, config in enumerate(combinations):
    run_name = f"Grid_Exp_{i}"
    print(f"\n=== Iniciando {run_name} ===")

    model = YOLO("yolov8n.pt")

    # ENTRENAMIENTO
    model.train(
        data="bccd.yaml",
        epochs=50,
        imgsz=640,
        name=run_name,
        project=OUTPUT_DIR,
        patience=10,
        device=0,
        verbose=False,
        **config
    )

    # VALIDACIÓN
    # Al validar, usamos project también para que guarde los plots de validación ahí mismo
    # Sin embargo, model.val() generalmente usa la ruta del modelo entrenado.
    # El objeto 'metrics' ya contiene los datos numéricos.
    metrics = model.val(split="val", verbose=False, project=OUTPUT_DIR, name=f"{run_name}_val")

    # Extraer métricas globales
    res = {
        'Experiment': run_name,
        **config,
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr
    }

    # Extraer métricas POR CLASE
    class_maps = metrics.box.maps
    for class_id, class_name in model.names.items():
        res[f"{class_name}_mAP95"] = class_maps[class_id]

    all_results.append(res)

    # Guardar CSV parcial en la ruta base (fuera de la carpeta runs para encontrarlo fácil)
    csv_path = BASE_PATH/ "resultados_parciales_gridsearch.csv"
    pd.DataFrame(all_results).to_csv(csv_path, index=False)

# RESULTADOS FINALES
df = pd.DataFrame(all_results)
df = df.sort_values(by="mAP50-95", ascending=False)
final_csv_path = BASE_PATH / "resultados_finales_gridsearch.csv"
df.to_csv(final_csv_path, index=False)

print(f"\nProceso finalizado. CSV guardado en: {final_csv_path}")


=== Iniciando Grid_Exp_0 ===
Ultralytics 8.3.233 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (Quadro RTX 3000, 5739MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=bccd.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=22, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Grid_Exp_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=10, perspective=0.0, plot

Exception in thread Thread-225 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/s.lopezmedina/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/home/s.lopezmedina/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s.lopezmedina/Maestria/4_computer_vision/codigo/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
  File "/home/s.lopezmedina/Maestria/4_computer_vision/codigo/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/s.lopezmedina/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/multiprocessing/queues

KeyboardInterrupt: 